In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings('ignore')

plt.style.use('ggplot')

In [2]:
cex_list = ['Bitmax', 'Binance', 'BingX', 'Bitbox', 
 'Bitfinex', 'BitGo', 'Bitkub', 'BitPanda',
 'Bitrue', 'Bitso', 'Bitstamp', 'Bittrex',
 'Bitvavo', 'BitZ', 'BKEX', 'Blockchain.com', 
 'BTC Markets', 'BTCTurk', 'ByBit', 'Changelly',
 'Coinbase', 'Coinex', 'Coinlist', 'CoinSpot',
 'Crypto.com', 'Gate.io', 'Hotbit', 'Indodax',
 'Koibanx', 'Korbit', 'Kraken', 'Kucoin',
 'MEXC', 'Netcoins', 'OkCoin', 'OKX',
 'Paribu', 'Phemex', 'ProBit', 'Ripio', 
 'UpHold', 'Vindax', 'WhiteBIT']



In [13]:
# Load the data into a DataFrame
df = pd.read_csv('CEX_balances/result.csv')

# Convert 'dt' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Pivot the DataFrame to have 'entity_name' as columns and 'dt' as the index
pivot_df = df.pivot(index='Date', columns='Entity', values='AlgoBalance')

# Create a new column for total balance by summing across entities
pivot_df['Total'] = pivot_df.sum(axis=1)
pivot_df.reset_index(inplace=True)

In [14]:
pivot_df

Entity,Date,BKEX,BTC Markets,BTCTurk,Binance,BingX,BitGo,BitPanda,BitZ,Bitbox,...,OKX,OkCoin,Paribu,Phemex,ProBit,Ripio,UpHold,Vindax,WhiteBIT,Total
0,2019-06-11,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000,0.000000e+00,0.0,0.000000,...,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.00000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00
1,2019-06-12,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000,0.000000e+00,0.0,0.000000,...,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.00000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00
2,2019-06-13,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000,0.000000e+00,0.0,0.000000,...,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.00000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00
3,2019-06-14,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000,0.000000e+00,0.0,0.000000,...,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.00000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00
4,2019-06-15,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000,0.000000e+00,0.0,0.000000,...,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.00000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1948,2024-10-10,35490.044553,1.507415e+07,1.843369e+07,5.644393e+08,100637.112879,0.198,1.196856e+08,0.1,4645.409785,...,4.608845e+07,873003.161901,1.023192e+07,1.632098e+06,31445.39049,7403.480497,7.139112e+08,538.566099,473668.677943,2.887029e+09
1949,2024-10-11,35490.044553,1.507405e+07,1.843369e+07,5.646042e+08,98168.705495,0.198,1.197758e+08,0.1,4645.409785,...,4.679610e+07,873003.161901,1.037864e+07,1.650201e+06,31445.39049,7372.260691,7.138955e+08,538.566099,473449.184345,2.894305e+09
1950,2024-10-12,35490.044553,1.507393e+07,1.843369e+07,5.648105e+08,85429.523849,0.198,1.197858e+08,0.1,4645.409785,...,4.656243e+07,873003.161901,1.037727e+07,1.676921e+06,31445.39049,7022.366519,7.139095e+08,538.566099,472579.244146,2.893204e+09
1951,2024-10-13,35490.044553,1.505112e+07,1.843369e+07,5.631157e+08,48518.845570,0.198,1.197326e+08,0.1,4645.409785,...,4.704884e+07,873003.161901,1.032753e+07,1.679673e+06,31171.22940,5635.828230,7.140634e+08,538.566099,472579.244146,2.889582e+09


In [15]:
pivot_df['Date'] = pd.to_datetime(pivot_df['Date'])
last_date = pivot_df['Date'].max()

# Step 3: Get the last day balances
last_day_df = pivot_df[pivot_df['Date'] == last_date]

# Step 4: Prepare ranking DataFrame
ranking_data = {'CEX': [], 'Balance': [], '24h%': [], '7d%': [], '30d%': [], '90d%': []}

# Get the list of CEX columns (excluding 'Date')
cex_columns = [col for col in pivot_df.columns if col != 'Date']

# Step 5: Populate the ranking data
for cex in cex_columns:
    # Extract the last day balance
    balance = last_day_df[cex].values[0]
    
    # Add CEX name and Balance
    ranking_data['CEX'].append(cex.replace(' Balance', ''))
    ranking_data['Balance'].append(balance)
    
    # Calculate 24-hour change
    previous_day = last_date - pd.Timedelta(days=1)
    previous_day_balance = pivot_df[pivot_df['Date'] == previous_day][cex]
    
    if not previous_day_balance.empty and previous_day_balance.values[0] != 0:
        change_24h = ((balance - previous_day_balance.values[0]) / previous_day_balance.values[0]) * 100
    else:
        change_24h = None
    ranking_data['24h%'].append(change_24h)
    
    # Calculate percentage changes for 7d, 30d, and 90d
    for days in [7, 30, 90]:
        past_date = last_date - pd.Timedelta(days=days)
        past_balance = pivot_df[pivot_df['Date'] == past_date][cex]
        
        if not past_balance.empty and past_balance.values[0] != 0:
            change = ((balance - past_balance.values[0]) / past_balance.values[0]) * 100
        else:
            change = None
        
        ranking_data[f'{days}d%'].append(change)

# Step 6: Create the ranking DataFrame
ranking_df = pd.DataFrame(ranking_data)

# Step 7: Sort the ranking DataFrame by Balance in descending order
ranking_df.sort_values(by='Balance', ascending=False, inplace=True)
ranking_df.reset_index(drop=True, inplace=True)
ranking_df.to_csv('data/CEX_ranking.csv', index=False)